Using pytorch quantization and pruning to prepare the model to be deployed on a microcontroller

In [1]:
!pip install ultralytics
!pip install roboflow

In [7]:
import torch
from torch.quantization import quantize_dynamic
import torchvision
from torchvision.models.detection import SSDLite320_MobileNet_V3_Large_Weights
from google.colab import drive
import torch.quantization
from roboflow import Roboflow

from ultralytics import YOLO

In [8]:
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:

weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
num_classes = 3
ssd_model = torchvision.models.detection.ssdlite320_mobilenet_v3_large(num_classes=num_classes)
preprocess = weights.transforms()

# Load the saved model weights
ssd_model.load_state_dict(torch.load('/content/gdrive/My Drive/MSc Project/Saved Models/ssd.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
# mount data set into drive
# change the api key and project name before uploading to github

rf = Roboflow(api_key="IYwf0SjALphOAs60cx0U")
project = rf.workspace("myworkspace-zkmzj").project("license-plate-detector-no-brand")
version = project.version(1)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.70, to fix: `pip install ultralytics==8.0.196`


In [11]:


drive_path = '/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n.pt'

yolo_model = YOLO(drive_path)



In [15]:
yolo_model.export(format="onnx", int8=True)


Ultralytics YOLOv8.2.70 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 2.0s, saved as '/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n.onnx' (11.7 MB)

Export complete (4.4s)
Results saved to /content/gdrive/My Drive/MSc Project/Saved Models
Predict:         yolo predict task=detect model=/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n.onnx imgsz=640 int8 
Validate:        yolo val task=detect model=/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n.onnx imgsz=640 data=data.yaml int8 
Visualize:       https://netron.app


'/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n.onnx'